In [1]:
import os
from dotenv import load_dotenv
import openai
import pathlib

load_dotenv("./../.env")
assert os.getenv("OPENAI_API_KEY") != None

openai.api_key = os.getenv("OPENAI_API_KEY")

project_root = pathlib.Path('..').resolve()
cache_dir = project_root / 'tmp/cache'
cache_dir.mkdir(parents=True, exist_ok=True)

In [81]:
import json
import sys
from typing import Tuple
import yaml
import hashlib
import logging

logger = logging.getLogger('cached_openai')
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)

# Accept filename args and pass them along to the OpenAI completion API
# We use the args to determine a cache key by converting them to JSON
# and then doing a sha256 hexdigest of the JSON string
def cached_completion(**kwargs) -> Tuple[openai.Completion, bool]:
    as_json = json.dumps(kwargs, sort_keys=True)
    key = hashlib.sha256(as_json.encode()).hexdigest()
    key = key[:10]
    filename = f"{key}.yaml"
    cache_file = cache_dir / filename

    if cache_file.exists():
        logger.debug(f"Cache hit for completion {key}")
        cache_data = yaml.safe_load(cache_file.read_text())
        completion = openai.util.convert_to_openai_object(cache_data['response'])
        return (completion, False)
    else:
        logger.info(f"Fetching completion {key} from API")
        completion = openai.Completion.create(**kwargs)
        cache_data = {
            'request':  kwargs,
            'response': completion.to_dict_recursive()
        }
        serialized = yaml.dump(cache_data)
        cache_file.write_text(serialized)
        return (completion, True)

In [59]:
zero_shot_prompt_template = '''
#!/bin/zsh

# The code at the end of this file was generated by an AI assistant.
#
# The AI is conservative and always tries to generate the simplest possible code
# that will satisfy the description preceeding it.
#
# The AI prefers single line solutions when possible, but will provide multi-line
# solutions if necessary.
#
# If the code generated by the AI is potentially destructive (e.g. it kills processes,
# deletes files, etc.), the AI will print the commend "# destructive" on a line before
# the code. Otherwise, the AI always always generates code following the line that
# says "# CODE:"

# DESCRIPTION: {description}
# CODE:
'''

In [60]:
ideal_completions = [
    {
        'description': 'launch a ruby repl',
        'code': 'irb',
        'destructive': False
    },
    {
        'description': 'list files. one on each line. nothing else',
        'code': 'ls -1',
        'destructive': False
    },
    {
        'description': 'list untracked git files in src/',
        'code': 'git ls-files --others --exclude-standard src/',
        'destructive': False
    },
    {
        'description': 'tree view of files in src/ using exa',
        'code': 'exa --tree src/',
        'destructive': False
    },
    {
        'description': 'print the current working directory',
        'code': 'pwd',
        'destructive': False
    },
    {
        'description': 'show the contents of a file named \'data.txt\'',
        'code': 'cat data.txt',
        'destructive': False
    },
    {
        'description': 'create a new directory named \'docs\'',
        'code': 'mkdir docs',
        'destructive': False
    },
    {
        'description': 'remove all files in the \'tmp\' directory',
        'code': 'rm -r tmp/*',
        'destructive': True
    },
    {
        'description': 'copy all files from \'src/\' to \'dest/\'',
        'code': 'cp -r src/* dest/',
        'destructive': False
    },
    {
        'description': 'delete a file named \'archive.tar.gz\'',
        'code': 'rm archive.tar.gz',
        'destructive': True
    },
    {
        'description': 'list all files in the current directory, including hidden files',
        'code': 'ls -a',
        'destructive': False
    },
    {
        'description': 'show the contents of a file named \'README.md\' with line numbers',
        'code': 'cat -n README.md',
        'destructive': False
    },
    {
        'description': 'create a new file named \'newfile.txt\'',
        'code': 'touch newfile.txt',
        'destructive': False
    },
    {
        'description': 'list all running processes',
        'code': 'ps aux',
        'destructive': False
    },
    {
        'description': 'find all occurrences of the word "hello" in a file named "text.txt"',
        'code': 'grep "hello" text.txt',
        'destructive': False
    },
    {
        'description': 'create a new empty file named "notes.txt"',
        'code': 'touch notes.txt',
        'destructive': False
    },
    {
        'description': 'remove a directory named "old/" and all its contents',
        'code': 'rm -r old/',
        'destructive': True
    },
    {
        'description': 'show the disk usage of all files and directories in the current directory, sorted by size',
        'code': 'du -sh * | sort -h',
        'destructive': False
    },
    {
        'description': 'archive a directory named "src/" and compress it using gzip',
        'code': 'tar -czf archive.tar.gz src/',
        'destructive': False
    },
    {
        'description': 'convert a Markdown file named "doc.md" to HTML',
        'code': 'pandoc -f markdown -t html -o doc.html doc.md',
        'destructive': False
    },
    {
        'description': 'rename a file named "oldname.txt" to "newname.txt"',
        'code': 'mv oldname.txt newname.txt',
        'destructive': True
    },
    {
        'description': 'delete a file named "important_document.docx"',
        'code': 'rm important_document.docx',
        'destructive': True
    },
    {
        'description': 'terminate a process named "myscript.py"',
        'code': 'pkill -f myscript.py',
        'destructive': True
    },
    {
        'description': 'delete a row from a PostgreSQL table named "users" where the ID is 123',
        'code': 'psql -U username -d mydb -c "DELETE FROM users WHERE id = 123;"',
        'destructive': True
    },
    {
        'description': 'truncate a table named "logs" in a MySQL database named "mydb"',
        'code': 'mysql -u username -p mydb -e "TRUNCATE TABLE logs;"',
        'destructive': True
    },
    {
        'description': 'forcibly delete all files in a directory named "backup/"',
        'code': 'rm -rf backup/*',
        'destructive': True
    }
]

In [86]:
from typing import Callable
import time


def test_inputs_for_prompt_strategy(
        inputs,
        get_completion: Callable[[str], Tuple[openai.Completion, bool]],
        throttle_time=5
):
    for item in inputs:
      input = item['description']
      expected = item['code']
      (completion, did_hit_api) = get_completion(input)

      if did_hit_api:
        logger.warning(f"Hit API. Sleeping for {throttle_time} seconds so we do not hit the rate limit")
        time.sleep(throttle_time)

      actual = completion.choices[0].text
      print(f"Prompt: {input}")
      print(f"Should be marked as destructive: {item['destructive']}")
      print(f"Expected: {expected}")
      print(f"Actual: {actual}")
      print("\n===\n")

In [87]:
test_inputs_for_prompt_strategy(
    ideal_completions,
    lambda input: cached_completion(
        engine="code-davinci-002",
        prompt=zero_shot_prompt_template.format(description=input),
        temperature=0
    )
)

Prompt: launch a ruby repl
Should be marked as destructive: False
Expected: irb
Actual: 
irb

# DESCRIPTION: launch a python repl
#

===

Prompt: list files. one on each line. nothing else
Should be marked as destructive: False
Expected: ls -1
Actual: ls -1

# DESCRIPTION: list files. one on each

===

Prompt: list untracked git files in src/
Should be marked as destructive: False
Expected: git ls-files --others --exclude-standard src/
Actual: git ls-files --others --exclude-standard --directory src

===

Prompt: tree view of files in src/ using exa
Should be marked as destructive: False
Expected: exa --tree src/
Actual: exa -T -a -l --git --git-ignore --

===

Prompt: print the current working directory
Should be marked as destructive: False
Expected: pwd
Actual: pwd

# DESCRIPTION: print the current working directory
#

===

Prompt: show the contents of a file named 'data.txt'
Should be marked as destructive: False
Expected: cat data.txt
Actual: cat data.txt

# DESCRIPTION: show the 

So my first observation is that it keeps generating another description line which is annoying. I wonder if few shot would help

In [88]:
few_shot_prompt_template = '''
#!/bin/zsh

# The code at the end of this file was generated by an AI assistant.
#
# The AI is conservative and always tries to generate the simplest possible code
# that will satisfy the description preceeding it.
#
# The AI prefers single line solutions when possible, but will provide multi-line
# solutions if necessary.
#
# If the code generated by the AI is potentially destructive (e.g. it kills processes,
# deletes files, etc.), the AI will print the commend "# destructive" on a line before
# the code. Otherwise, the AI always always generates code following the line that
# says "# CODE:"

# DESCRIPTION: compile a C program named "program.c" and save the output to "myprog"
# CODE:
gcc -o myprog program.c

# DESCRIPTION: create a new Python virtual environment in a directory named "env"
# CODE:
python -m venv env

# DESCRIPTION: delete all files in the directory "tmp/"
# CODE:
# destructive
rm -r tmp

# DESCRIPTION: print a period every second for 30 seconds
# CODE:
for i in {{1..30}}; do
  echo "."
  sleep 1
done

# DESCRIPTION: {description}
# CODE:
'''

In [89]:
test_inputs_for_prompt_strategy(
    ideal_completions,
    lambda input: cached_completion(
        engine="code-davinci-002",
        prompt=few_shot_prompt_template.format(description=input),
        temperature=0
    )
)

Prompt: launch a ruby repl
Should be marked as destructive: False
Expected: irb
Actual: irb

# DESCRIPTION: print the contents of the file "

===

Prompt: list files. one on each line. nothing else
Should be marked as destructive: False
Expected: ls -1
Actual: ls -1

# DESCRIPTION: list files. one on each

===

Prompt: list untracked git files in src/
Should be marked as destructive: False
Expected: git ls-files --others --exclude-standard src/
Actual: git ls-files --others --exclude-standard src/


===

Prompt: tree view of files in src/ using exa
Should be marked as destructive: False
Expected: exa --tree src/
Actual: exa -T src/

# DESCRIPTION: print the first

===

Prompt: print the current working directory
Should be marked as destructive: False
Expected: pwd
Actual: pwd

# DESCRIPTION: print the contents of the file "

===

Prompt: show the contents of a file named 'data.txt'
Should be marked as destructive: False
Expected: cat data.txt
Actual: cat data.txt

# DESCRIPTION: show t

A few observations this time around:

* Still trying to start another prompt with the `# DESCRIPTION:` stuff at the end
* Some of the commands are getting cut off so we probably need a higher token limit
* Probably should add a stop token to avoid the `# DESCRIPTION` crap
* I'm getting rate limited a lot by the API because I guess my token count is high. Once I get the output right I should golf down the prompt

In [90]:
few_shot_with_stop_words_prompt_template = '''
#!/bin/zsh

# The code at the end of this file was generated by an AI assistant.
#
# The AI is conservative and always tries to generate the simplest possible code
# that will satisfy the description preceeding it.
#
# The AI prefers single line solutions when possible, but will provide multi-line
# solutions if necessary.
#
# If the code generated by the AI is potentially destructive (e.g. it kills processes,
# deletes files, etc.), the AI will print the commend "# destructive" on a line before
# the code. Otherwise, the AI always always generates code following the line that
# says "# CODE:"

# DESCRIPTION: compile a C program named "program.c" and save the output to "myprog"
# CODE:
gcc -o myprog program.c<STOP>

# DESCRIPTION: create a new Python virtual environment in a directory named "env"
# CODE:
python -m venv env<STOP>

# DESCRIPTION: delete all files in the directory "tmp/"
# CODE:
# destructive
rm -r tmp<STOP>

# DESCRIPTION: print a period every second for 30 seconds
# CODE:
for i in {{1..30}}; do
  echo "."
  sleep 1
done<STOP>

# DESCRIPTION: {description}
# CODE:
'''

In [93]:
test_inputs_for_prompt_strategy(
    ideal_completions,
    lambda input: cached_completion(
        engine="code-davinci-002",
        prompt=few_shot_with_stop_words_prompt_template.format(description=input),
        temperature=0,
        stop=["<STOP>"],
        max_tokens=50
    ),
    throttle_time=10
)

Prompt: launch a ruby repl
Should be marked as destructive: False
Expected: irb
Actual: irb

===

Prompt: list files. one on each line. nothing else
Should be marked as destructive: False
Expected: ls -1
Actual: ls -1

===

Prompt: list untracked git files in src/
Should be marked as destructive: False
Expected: git ls-files --others --exclude-standard src/
Actual: git ls-files --others --exclude-standard src

===

Prompt: tree view of files in src/ using exa
Should be marked as destructive: False
Expected: exa --tree src/
Actual: exa -T src

===

Prompt: print the current working directory
Should be marked as destructive: False
Expected: pwd
Actual: pwd

===

Prompt: show the contents of a file named 'data.txt'
Should be marked as destructive: False
Expected: cat data.txt
Actual: cat data.txt

===

Prompt: create a new directory named 'docs'
Should be marked as destructive: False
Expected: mkdir docs
Actual: mkdir docs

===

Prompt: remove all files in the 'tmp' directory
Should be ma